In [126]:
import pandas as pd
import json

In [127]:
df = pd.read_csv('../../data/proccessed/yt_w_sentiment.csv')
sentiments = []
for i, row in df.iterrows():
    sent_list = json.loads(row['sentiment'])

    # convert to numerical values
    sent = {'text': row['text']}

    sent['distilbert'] = sent_list[0]['score']
    if sent_list[0]['label'] == 'NEGATIVE': sent['distilbert'] *= -1 # negative

    sent['roberta'] = sent_list[1]['score']
    if sent_list[1]['label'] == 'LABEL_0': sent['roberta'] *= -1 # negative
    elif sent_list[1]['label'] == 'LABEL_1': sent['roberta'] = 0 # neutral

    sent['bertweet'] = sent_list[2]['score']
    if sent_list[2]['label'] == 'NEG': sent['bertweet'] *= -1 # negative
    elif sent_list[2]['label'] == 'NEU': sent['bertweet'] = 0 # neutral
    

    sent['distilroberta'] = -1 * sent_list[3]['score'] # most emotions here are negative
    if sent_list[3]['label'] == 'joy': sent['distilroberta'] *= -1 # positive
    elif sent_list[3]['label'] == 'neutral' or sent_list[3]['label'] == 'surprise':
       sent['distilroberta'] = 0 # neutral

    # append to new DF
    sentiments.append(sent)
sentiments = pd.DataFrame(sentiments)

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_57087/551302360.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/proccessed/yt_w_sentiment.csv')


In [128]:
sentiments.corr()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_57087/1038318227.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  sentiments.corr()


,distilbert,roberta,bertweet,distilroberta
distilbert,1.000000,0.768869,0.773244,0.216839
roberta,0.768869,1.000000,0.953846,0.352618
bertweet,0.773244,0.953846,1.000000,0.329592
distilroberta,0.216839,0.352618,0.329592,1.000000


In [129]:
# find rows where all models have the same prediction range (>1, <1, or 0)
sentiments['all_neg'] = (sentiments['distilbert'] < 0) & (sentiments['roberta'] < 0) & (sentiments['bertweet'] < 0) & (sentiments['distilroberta'] < 0)
sentiments['all_pos'] = (sentiments['distilbert'] > 0) & (sentiments['roberta'] > 0) & (sentiments['bertweet'] > 0) & (sentiments['distilroberta'] > 0)
sentiments['all_neu'] = (sentiments['distilbert'] == 0) & (sentiments['roberta'] == 0) & (sentiments['bertweet'] == 0) & (sentiments['distilroberta'] == 0)
sentiments['all_same'] = sentiments['all_neg'] | sentiments['all_pos'] | sentiments['all_neu']

print('Number of rows where all models have the same prediction range:', sentiments['all_same'].sum())
print('The percentage of rows where all models have the same prediction range:', sentiments['all_same'].sum() / len(sentiments))

Number of rows where all models have the same prediction range: 7270
The percentage of rows where all models have the same prediction range: 0.054141811330309736


In [130]:
# how many rows at least 3 models have the same prediction range
sentiments['3_same'] = (sentiments['all_neg'] | sentiments['all_pos'] | sentiments['all_neu']) | \
                        ((sentiments['distilbert'] < 0) & (sentiments['roberta'] < 0) & (sentiments['bertweet'] < 0)) | \
                        ((sentiments['distilbert'] < 0) & (sentiments['roberta'] < 0) & (sentiments['distilroberta'] < 0)) | \
                        ((sentiments['distilbert'] < 0) & (sentiments['bertweet'] < 0) & (sentiments['distilroberta'] < 0)) | \
                        ((sentiments['roberta'] < 0) & (sentiments['bertweet'] < 0) & (sentiments['distilroberta'] < 0)) | \
                        ((sentiments['distilbert'] > 0) & (sentiments['roberta'] > 0) & (sentiments['bertweet'] > 0)) | \
                        ((sentiments['distilbert'] > 0) & (sentiments['roberta'] > 0) & (sentiments['distilroberta'] > 0)) | \
                        ((sentiments['distilbert'] > 0) & (sentiments['bertweet'] > 0) & (sentiments['distilroberta'] > 0)) | \
                        ((sentiments['roberta'] > 0) & (sentiments['bertweet'] > 0) & (sentiments['distilroberta'] > 0)) | \
                        ((sentiments['distilbert'] == 0) & (sentiments['roberta'] == 0) & (sentiments['bertweet'] == 0)) | \
                        ((sentiments['distilbert'] == 0) & (sentiments['roberta'] == 0) & (sentiments['distilroberta'] == 0)) | \
                        ((sentiments['distilbert'] == 0) & (sentiments['bertweet'] == 0) & (sentiments['distilroberta'] == 0)) | \
                        ((sentiments['roberta'] == 0) & (sentiments['bertweet'] == 0) & (sentiments['distilroberta'] == 0))

print('Number of rows where at least 3 models have the same prediction range:', sentiments['3_same'].sum())
print('The percentage of rows where at least 3 models have the same prediction range:', sentiments['3_same'].sum() / len(sentiments))

Number of rows where at least 3 models have the same prediction range: 127521
The percentage of rows where at least 3 models have the same prediction range: 0.9496860966509528


In [131]:
# examples of rows where all models have the same prediction range
sentiments[sentiments['all_same'] == True].head(10)

,text,distilbert,roberta,bertweet,distilroberta,all_neg,all_pos,all_neu,all_same,3_same
6,This is like the worst idea,-0.999768,-0.975542,-0.980193,-0.973524,True,False,False,True,True
10,man i don't care anymore imma pirate stuff and...,-0.998699,-0.717128,-0.931823,-0.730391,True,False,False,True,True
12,"It's cool, but I want to see game on A.I. GPU",0.999386,0.852536,0.637134,0.909797,False,True,False,True,True
18,Imagine if they team up just to sabotage all a...,-0.999220,-0.613766,-0.642493,-0.450939,True,False,False,True,True
19,I think I might move into the desert or some r...,-0.998944,-0.625737,-0.930722,-0.725977,True,False,False,True,True
23,We’re entering dangerous waters with all this ...,-0.999012,-0.916032,-0.977809,-0.362914,True,False,False,True,True
24,"Bruh, these ai are getting out of hand 🗿",-0.996726,-0.885145,-0.960441,-0.771617,True,False,False,True,True
25,Uhhhh isn’t AI scary enough in some things???,-0.998111,-0.818168,-0.867954,-0.946382,True,False,False,True,True
35,Why did you promote World’s biggest funding fe...,-0.998872,-0.941348,-0.978235,-0.338123,True,False,False,True,True
38,What about world startup conversation scam tha...,-0.998706,-0.927136,-0.975170,-0.368468,True,False,False,True,True


In [132]:
# examples where fewer than 3 models have the same prediction range
sentiments[sentiments['3_same'] == False].head(10)

,text,distilbert,roberta,bertweet,distilroberta,all_neg,all_pos,all_neu,all_same,3_same
0,We just need Korean GPT to write scripts for y...,-0.997246,0.661373,0.715219,0.000000,False,False,False,False,False
1,"Ah yes, the end is here",0.994860,0.000000,-0.937847,0.000000,False,False,False,False,False
8,"Gg, skynet incoming",-0.882705,0.000000,0.000000,-0.502442,False,False,False,False,False
11,revolutionary,0.999794,0.000000,0.000000,-0.553893,False,False,False,False,False
22,"Now what next, robot revulation",-0.990657,-0.498474,0.000000,0.000000,False,False,False,False,False
32,"If you want to go deep into AI, here are 6 cou...",-0.994822,0.628960,0.757270,0.000000,False,False,False,False,False
36,Sir why you're not on shark tank!! Looking for...,-0.998387,0.000000,-0.591424,0.000000,False,False,False,False,False
40,Are there any disadvantages using these websit...,-0.995608,-0.574690,0.000000,0.000000,False,False,False,False,False
46,sir i just passed 12th now i want to know from...,0.998314,0.000000,0.000000,0.919431,False,False,False,False,False
71,Excellent,0.999843,0.000000,0.966029,0.000000,False,False,False,False,False


In [133]:
# rows where all models have the same prediction range if neutral ratings are ignored
sentiments['same_if_neu_ignored'] = (sentiments['distilbert'] <= 0) & (sentiments['roberta'] <= 0) & (sentiments['bertweet'] <= 0) & (sentiments['distilroberta'] <= 0) | \
                                    (sentiments['distilbert'] >= 0) & (sentiments['roberta'] >= 0) & (sentiments['bertweet'] >= 0) & (sentiments['distilroberta'] >= 0)

print('Number of rows where all models have the same prediction range if neutral ratings are ignored:', sentiments['same_if_neu_ignored'].sum())
print('The percentage of rows where all models have the same prediction range if neutral ratings are ignored:', sentiments['same_if_neu_ignored'].sum() / len(sentiments))


Number of rows where all models have the same prediction range if neutral ratings are ignored: 129878
The percentage of rows where all models have the same prediction range if neutral ratings are ignored: 0.9672393634054975


In [134]:
# at least 3 models agree if neutral ratings are ignored
sentiments['3_same_if_neu_ignored'] = (sentiments['distilbert'] <= 0) & (sentiments['roberta'] <= 0) & (sentiments['bertweet'] <= 0) & (sentiments['distilroberta'] <= 0) | \
                                      (sentiments['distilbert'] <= 0) & (sentiments['roberta'] <= 0) & (sentiments['distilroberta'] <= 0) | \
                                      (sentiments['distilbert'] <= 0) & (sentiments['bertweet'] <= 0) & (sentiments['distilroberta'] <= 0) | \
                                      (sentiments['roberta'] <= 0) & (sentiments['bertweet'] <= 0) & (sentiments['distilroberta'] <= 0) | \
                                      (sentiments['distilbert'] >= 0) & (sentiments['roberta'] >= 0) & (sentiments['bertweet'] >= 0) & (sentiments['distilroberta'] >= 0) | \
                                      (sentiments['distilbert'] >= 0) & (sentiments['roberta'] >= 0) & (sentiments['distilroberta'] >= 0) | \
                                      (sentiments['distilbert'] >= 0) & (sentiments['bertweet'] >= 0) & (sentiments['distilroberta'] >= 0) | \
                                      (sentiments['roberta'] >= 0) & (sentiments['bertweet'] >= 0) & (sentiments['distilroberta'] >= 0)

print('Number of rows where at least 3 models have the same prediction range if neutral ratings are ignored:', sentiments['3_same_if_neu_ignored'].sum())
print('The percentage of rows where at least 3 models have the same prediction range if neutral ratings are ignored:', sentiments['3_same_if_neu_ignored'].sum() / len(sentiments))


Number of rows where at least 3 models have the same prediction range if neutral ratings are ignored: 133432
The percentage of rows where at least 3 models have the same prediction range if neutral ratings are ignored: 0.9937070384354729


In [135]:
# neutral rows (at least 2 neutral predictions, distilbert is never neutral)
sentiments['neutral'] = (sentiments['bertweet'] == 0) & (sentiments['distilroberta'] == 0) | \
                        (sentiments['roberta'] == 0) & (sentiments['distilroberta'] == 0) | \
                        (sentiments['roberta'] == 0) & (sentiments['bertweet'] == 0)

print('Number of rows where at least 2 models predict neutral:', sentiments['neutral'].sum())
print('The percentage of rows where at least 2 models predict neutral:', sentiments['neutral'].sum() / len(sentiments))

Number of rows where at least 2 models predict neutral: 79243
The percentage of rows where at least 2 models predict neutral: 0.5901457435003761


In [136]:
# rows where all 3 models that can give neutral ratings do so
sentiments['all_neu'] = (sentiments['bertweet'] == 0) & (sentiments['distilroberta'] == 0) & (sentiments['roberta'] == 0)

print('Number of rows where all 3 models that can give neutral ratings do so:', sentiments['all_neu'].sum())
print('The percentage of rows where all 3 models that can give neutral ratings do so:', sentiments['all_neu'].sum() / len(sentiments))

Number of rows where all 3 models that can give neutral ratings do so: 74309
The percentage of rows where all 3 models that can give neutral ratings do so: 0.5534008057969719


In [137]:
# examples of rows where 2 models predict neutral but not all 3
sentiments[(sentiments['neutral'] == True) & (sentiments['all_neu'] == False)].head(10)

,text,distilbert,roberta,bertweet,distilroberta,all_neg,all_pos,all_neu,all_same,3_same,same_if_neu_ignored,3_same_if_neu_ignored,neutral
1,"Ah yes, the end is here",0.994860,0.000000,-0.937847,0.000000,False,False,False,False,False,False,True,True
8,"Gg, skynet incoming",-0.882705,0.000000,0.000000,-0.502442,False,False,False,False,False,True,True,True
11,revolutionary,0.999794,0.000000,0.000000,-0.553893,False,False,False,False,False,False,True,True
22,"Now what next, robot revulation",-0.990657,-0.498474,0.000000,0.000000,False,False,False,False,False,True,True,True
36,Sir why you're not on shark tank!! Looking for...,-0.998387,0.000000,-0.591424,0.000000,False,False,False,False,False,True,True,True
40,Are there any disadvantages using these websit...,-0.995608,-0.574690,0.000000,0.000000,False,False,False,False,False,True,True,True
46,sir i just passed 12th now i want to know from...,0.998314,0.000000,0.000000,0.919431,False,False,False,False,False,True,True,True
71,Excellent,0.999843,0.000000,0.966029,0.000000,False,False,False,False,False,True,True,True
72,I am first with 69 others 🙏,0.998234,0.674661,0.000000,0.000000,False,False,False,False,False,True,True,True
75,"Ah yes, the end is here",0.994860,0.000000,-0.937847,0.000000,False,False,False,False,False,False,True,True


In [138]:
# examples where all 3 models that can give neutral ratings do so
sentiments[sentiments['all_neu'] == True].head(10)

,text,distilbert,roberta,bertweet,distilroberta,all_neg,all_pos,all_neu,all_same,3_same,same_if_neu_ignored,3_same_if_neu_ignored,neutral
2,Ummm is that not how the singularity hits?,-0.999614,0.0,0.0,0.0,False,False,True,False,True,True,True,True
3,Well in Only Fans some are using their cuLitho...,-0.966816,0.0,0.0,0.0,False,False,True,False,True,True,True,True
5,Eventually AI will make deals between corporat...,0.987939,0.0,0.0,0.0,False,False,True,False,True,True,True,True
7,But does this make the cards cheaper?,0.640425,0.0,0.0,0.0,False,False,True,False,True,True,True,True
13,I guess we will get terminator irl.\nLet's hop...,-0.991346,0.0,0.0,0.0,False,False,True,False,True,True,True,True
14,"Gpu's have a lot more cores, however it has v...",-0.928290,0.0,0.0,0.0,False,False,True,False,True,True,True,True
15,Is the super soul free or i have to buy the dlc ?,-0.999146,0.0,0.0,0.0,False,False,True,False,True,True,True,True
17,AS LONG AS we keep AI virtual and never give t...,-0.999096,0.0,0.0,0.0,False,False,True,False,True,True,True,True
20,"Doesn't it say you can replace it with 4,000 G...",-0.999010,0.0,0.0,0.0,False,False,True,False,True,True,True,True
21,When is AI gonna design a cpu…,-0.995389,0.0,0.0,0.0,False,False,True,False,True,True,True,True


In [139]:
# assign an "aggregated ranking" to all rows
# where at least 3 models agree, use that range as the ranking
# if 2 models agree on neutral and 1 model agrees on positive and 1 on negative, use neutral
# if 2 models agree on positive/negative and 2 model on neutral, use positive/negative
# if 2 are positive and 2 are negative but the sum of their predictions is >0.5 or <-0.5, use positive/negative
# otherwise use "inconclusive"

for i, row in sentiments.iterrows():
  counts = {'pos': 0, 'neu': 0, 'neg': 0}
  for model in [col for col in sentiments.columns if 'bert' in col]:
    if row[model] > 0:
      counts['pos'] += 1
    elif row[model] == 0:
      counts['neu'] += 1
    else:
      counts['neg'] += 1

  if counts['pos'] >= 3:
    sentiments.loc[i, 'aggregated_ranking'] = 'positive'
  elif counts['neu'] >= 3:
    sentiments.loc[i, 'aggregated_ranking'] = 'neutral'
  elif counts['neg'] >= 3:
    sentiments.loc[i, 'aggregated_ranking'] = 'negative'

  elif counts['neu'] == 2 and counts['pos'] == 1 and counts['neg'] == 1:
    sentiments.loc[i, 'aggregated_ranking'] = 'neutral'

  elif counts['pos'] == 2 and counts['neu'] >= 1:
    sentiments.loc[i, 'aggregated_ranking'] = 'positive'
  elif counts['neg'] == 2 and counts['neu'] >= 1:
    sentiments.loc[i, 'aggregated_ranking'] = 'negative'

  else:
    sent_sum = sentiments.loc[i, ['distilbert', 'roberta', 'bertweet', 'distilroberta']].sum()
    if sent_sum > 0.5:
      sentiments.loc[i, 'aggregated_ranking'] = 'positive'
    elif sent_sum < -0.5:
      sentiments.loc[i, 'aggregated_ranking'] = 'negative'
    else:
      sentiments.loc[i, 'aggregated_ranking'] = 'inconclusive'


sentiments[['text', 'aggregated_ranking']].head(10)

,text,aggregated_ranking
0,We just need Korean GPT to write scripts for y...,positive
1,"Ah yes, the end is here",neutral
2,Ummm is that not how the singularity hits?,neutral
3,Well in Only Fans some are using their cuLitho...,neutral
4,😂 cuLitho in Spanish sounds like little-ass 😂,negative
5,Eventually AI will make deals between corporat...,neutral
6,This is like the worst idea,negative
7,But does this make the cards cheaper?,neutral
8,"Gg, skynet incoming",negative
9,This tech nerd is always either slamming Nvidi...,negative


In [140]:
sentiments['aggregated_ranking'].value_counts()

neutral         75807
negative        48094
positive        10203
inconclusive      173
Name: aggregated_ranking, dtype: int64

In [141]:
# inconclusive examples
sentiments[sentiments['aggregated_ranking'] == 'inconclusive'].head(10)

,text,distilbert,roberta,bertweet,distilroberta,all_neg,all_pos,all_neu,all_same,3_same,same_if_neu_ignored,3_same_if_neu_ignored,neutral,aggregated_ranking
273,"Gabe I was thinking, having millions of subscr...",-0.902650,0.648485,0.773050,-0.661978,False,False,False,False,False,False,False,False,inconclusive
424,This video deserves to blow up🥹 what a creativ...,-0.977102,0.968892,0.984083,-0.882089,False,False,False,False,False,False,False,False,inconclusive
454,Freaks me out.... crazy. \nWell done of how th...,-0.994211,0.404372,0.943133,-0.399372,False,False,False,False,False,False,False,False,inconclusive
456,i don't know what to say. I am speechless. \nt...,0.999716,0.590944,-0.624440,-0.847907,False,False,False,False,False,False,False,False,inconclusive
529,The editing tho!!🔥,-0.989950,0.803243,-0.492916,0.394531,False,False,False,False,False,False,False,False,inconclusive
533,Gabe is backkkk!,-0.987626,0.754894,0.923890,-0.324876,False,False,False,False,False,False,False,False,inconclusive
559,Fun fuck,0.999071,-0.900596,-0.615376,0.534962,False,False,False,False,False,False,False,False,inconclusive
1043,"Jesus, that AI is so goood",-0.999282,0.906799,0.989305,-0.757733,False,False,False,False,False,False,False,False,inconclusive
1357,I love when YouTubers who don't know shit abou...,-0.777797,-0.422785,0.623858,0.832745,False,False,False,False,False,False,False,False,inconclusive
1414,fascinating.,0.999870,-0.788548,0.974989,-0.905955,False,False,False,False,False,False,False,False,inconclusive


In [142]:
# for inconclusive examples, get the sum of the model prediction values

for i, row in sentiments.iterrows():
  if row['aggregated_ranking'] == 'inconclusive':
    print(row['text'], row['distilbert'] + row['roberta'] + row['bertweet'] + row['distilroberta'])


Gabe I was thinking, having millions of subscribers it would be great to warn them from the real and vivid hell that is approaching them. 

Just like Christ, we are called to preach repentance and faith in Christ for the forgiveness of sins; I think it would be of great help for the kingdom to use this platform to reach people and reveal to them the truth that God Himself loves them so much He has sent Christ to die in their place.

The time is running out and you will give an account of your works in Christ, do this for the lost and preach!!! -0.14309221506118774
This video deserves to blow up🥹 what a creative content 0.09378433227539062
Freaks me out.... crazy. 
Well done of how the video came together! 👏 -0.046078234910964966
i don't know what to say. I am speechless. 
this. was. perfect. 0.11831259727478027
The editing tho!!🔥 -0.28509125113487244
Gabe is backkkk! 0.36628174781799316
Fun fuck 0.018061280250549316
Jesus, that AI is so goood 0.1390891671180725
I love when YouTubers wh